In [1]:
import pandas as pd
import numpy as np
import NB_Funcs as hlp
from scipy.ndimage import convolve
import pickle

# Autocorrelated K model

$\frac{dN}{dt} = r * N(1-\frac{N}{K(x)})$ 

$\frac{dN}{dt} = r * N - \frac{r * N^2}{K(x)}$

where $x$ is distance to border. 

$\frac{dN}{dt*N} = r - \frac{rN}{K(x)}$

$ K(x) = rN / (r - \frac{dN}{dtN}) $

In [2]:
Ks = np.load('../data/RawData/ACK/ACK_K.npy')
N_K = np.load('../data/RawData/ACK/ACK_N_curves.npy')

N_K = N_K[:,:250,:,:] # we cut it off as all colonies have already gone into stationary phase

N_K_stnd = np.array(N_K, copy = True)

dt = 0.1

time = np.arange(N_K.shape[1])*dt
plates = N_K.shape[0]
t = N_K.shape[1]
rows = N_K.shape[2]
cols = N_K.shape[3]

plate_maximums = {}

for p in range(plates):
    plate_maximums[p] = np.max(N_K[p,:,:,:])
    N_K_stnd[p,:,:,:] = N_K_stnd[p,:,:,:] / np.max(N_K[p,:,:,:])
    
print(Ks.shape)
print(N_K.shape)
print(N_K_stnd.shape)

(32, 48)
(4, 250, 32, 48)
(4, 250, 32, 48)


In [3]:
# 32x48 array of ring value
rings = np.empty((rows,cols))

edge_r = (0,rows-1)
edge_c = (0,cols-1)

for i in range(rows):
    for j in range(cols):
        
        rings[i,j] = min(abs(i-edge_r[0])+1,abs(i-edge_r[1])+1,
                             abs(j-edge_c[0])+1,abs(j-edge_c[1])+1)

# Preprocessing

In [4]:
#2 different ways of storing the data, ring format and colony format

ring_form_data = {} # {Plate_Num: {Ring_Num: List[DFs]}}

colony_form_data = {} # {Plate_Num: List[32 * List[48 * DF]]}

## Colony Format

In [5]:
for p in range(plates):
    colony_form_data[p] = []
    
    for i in range(rows):
        for j in range(cols):
            Ps = np.ones((t)) * p
            Is = np.ones((t)) * i
            Js = np.ones((t)) * j
            N_init = np.ones((t)) * N_K[p,0,i,j]
            N = N_K[p,:,i,j]
            N_init_stnd = np.ones((t)) * N_K_stnd[p,0,i,j]
            N_stnd = N_K_stnd[p,:,i,j]
            K = np.ones((t)) * Ks[i][j]
            
            data = np.vstack([time,Ps,Is,Js,N_init,N,N_init_stnd,N_stnd,K]).T
            df = pd.DataFrame(data = data, columns = ["time","plate","i","j","N_init","Pop",
                                                      "N_init_stnd","Pop_stnd","K"])
            if j == 0:
                colony_form_data[p].append([df])
            else: colony_form_data[p][i].append(df)
            
for p in range(plates):
    dNdt = hlp.cubic_splines(N_K[p],time)
    dNdt_stnd = hlp.cubic_splines(N_K_stnd[p],time)
    for i in range(rows):
        for j in range(cols):
            colony_form_data[p][i][j]['dNdt'] = dNdt[:,i,j]
            colony_form_data[p][i][j]['dNdt_stnd'] = dNdt_stnd[:,i,j]
            
for p in range(plates):
    Nbar = np.empty((t,rows,cols))

    kernel = np.array([[1,1,1],
                       [1,0,1],
                       [1,1,1]])
    
    #neighbors = convolve(np.ones((32,48)),kernel,mode='constant')
    
    for tic in range(t):

        sums = convolve(N_K[p,tic,:,:],kernel,mode='constant')
        
        Nbar[tic,:,:] = sums / 8 #we deem every colony to have 8 neighbors to reflect differences in averages
                                    #between border colonies and others

    for i in range(rows):
        for j in range(cols):
            colony_form_data[p][i][j]['Nbar'] = Nbar[:,i,j]
            colony_form_data[p][i][j]['Cum_Nbar'] = np.cumsum(Nbar[:,i,j])*dt
            colony_form_data[p][i][j]['Cum_N'] = np.cumsum(N_K[p,:,i,j])*dt
            
            colony_form_data[p][i][j]['Nbar_stnd'] = Nbar[:,i,j] / np.max(N_K)
            colony_form_data[p][i][j]['Cum_Nbar_stnd'] = np.cumsum(Nbar[:,i,j])*dt / np.max(N_K)
            colony_form_data[p][i][j]['Cum_N_stnd'] = np.cumsum(N_K[p,:,i,j])*dt / np.max(N_K)
            
            

for p in range(plates):
    for i in range(rows):
        for df in colony_form_data[p][i]:
            df['dNovN'] = df['dNdt'] / df['Pop']
            df['dNovN_stnd'] = df['dNdt_stnd'] / df['Pop_stnd']

for p in range(plates):
    for i in range(rows):
        for j in range(cols):
            colony_form_data[p][i][j]['Ring'] = np.ones((t)) * rings[i,j]
    
colony_form_data[0][0][0].head(3)

,time,plate,i,j,N_init,Pop,N_init_stnd,Pop_stnd,K,dNdt,dNdt_stnd,Nbar,Cum_Nbar,Cum_N,Nbar_stnd,Cum_Nbar_stnd,Cum_N_stnd,dNovN,dNovN_stnd,Ring
0,0.0,0.0,0.0,0.0,122705.342189,122705.342189,0.012272,0.012272,10000000.0,59169.183589,0.005918,45512.986141,4551.298614,12270.534219,0.004552,0.000455,0.001227,0.482205,0.482205,1.0
1,0.1,0.0,0.0,0.0,122705.342189,128765.326294,0.012272,0.012878,10000000.0,62053.618932,0.006206,47759.894777,9327.288092,25147.066848,0.004777,0.000933,0.002515,0.481912,0.481912,1.0
2,0.2,0.0,0.0,0.0,122705.342189,135120.690062,0.012272,0.013514,10000000.0,65076.776868,0.006508,50116.241385,14338.912230,38659.135855,0.005012,0.001434,0.003866,0.481620,0.481620,1.0


## Ring Format

In [6]:
for p in range(plates):
    checklist = {}
    for i in range(rows):
        for df in colony_form_data[p][i]:
            ring_val = df['Ring'][0]
            if ring_val in checklist:
                checklist[ring_val].append(df)
            else:
                checklist[ring_val] = [df]
    
    ring_form_data[p] = checklist
    
print(len(ring_form_data[0][16]))
ring_form_data[0][1][0].head(3)

36


,time,plate,i,j,N_init,Pop,N_init_stnd,Pop_stnd,K,dNdt,dNdt_stnd,Nbar,Cum_Nbar,Cum_N,Nbar_stnd,Cum_Nbar_stnd,Cum_N_stnd,dNovN,dNovN_stnd,Ring
0,0.0,0.0,0.0,0.0,122705.342189,122705.342189,0.012272,0.012272,10000000.0,59169.183589,0.005918,45512.986141,4551.298614,12270.534219,0.004552,0.000455,0.001227,0.482205,0.482205,1.0
1,0.1,0.0,0.0,0.0,122705.342189,128765.326294,0.012272,0.012878,10000000.0,62053.618932,0.006206,47759.894777,9327.288092,25147.066848,0.004777,0.000933,0.002515,0.481912,0.481912,1.0
2,0.2,0.0,0.0,0.0,122705.342189,135120.690062,0.012272,0.013514,10000000.0,65076.776868,0.006508,50116.241385,14338.912230,38659.135855,0.005012,0.001434,0.003866,0.481620,0.481620,1.0


## Functions for Storing and Loading Results

In [7]:
def pickle_save(filename, data):
    with open(filename,'wb') as _save:
        pickle.dump(data,_save)
    
def pickle_load(filename):
    with open(filename,'rb') as _load:
        var = pickle.load(_load)
    return var

In [8]:
pickle_save('../Data/Train_Test/ACK_PreprocessedData',(colony_form_data,ring_form_data,plate_maximums))